<a href="https://colab.research.google.com/github/Cirediallo/DLIL/blob/main/Copy_of_05_Classification_de_textes_real.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification de textes

La **classification** consiste à attribuer une classe à chaque texte (objet, instance, point) à classer. On parle de *classification binaire* (_binary classification_) quand il y a deux classes exclusives. On parle de *classification en classes multiples* (_multiclass classification_) pour désigner la répartition d'un lot de textes entre plus de deux ensembles (ou classes), une classe par instance. On parle de *classification multi-étiquettes* (_multi-label classification_) pour désigner les problèmes de classification où plusieurs étiquettes (classes) peuvent être assignées à une même instance.

Reconnaître si un email est un spam, si une photo contient une voiture... sont des problèmes de classification (binaire).


# Sentiment analysis as a classification task


La tâche classique d'analyse de sentiment consiste à annoter un texte donné selon une polarité positive ou négative exprimée dans le texte.


## QUESTION

* Selon vous la tâche d'analyse de sentiment tel que définie juste au dessus peut se définir comme un problème de 1) classification binaire, 2) classification en classes multiples ou 3) en classification multi-étiquettes ?

## VOTRE REPONSE

According to the definition above, the sentiment analysis classification can be defined as binary classification


# Configuration de l'environnement Google Colab en mode GPU


Quel mode d'exécution utilisez-vous (via GPU ou via CPU) ? Que vous dis le code ci-dessous ?

This code below try to allocate GPU and check the allocated amount of GPU by Colab 

In [1]:
# memory footprint support libraries/code
# https://medium.com/@oribarel/getting-the-most-out-of-your-google-colab-2b0585f82403
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed

print(GPUs)

def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
#

if len(GPUs) >0: 
  gpu = GPUs[0]
  printm()
else:
  print ('no GPU. Are you sure the hardware accelerator is configured to GPU? To do this go to Runtime→Change runtime type and change the Hardware accelerator to GPU.') 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=cfdfefe51d564ca563b829539388b046cdfc74fe7ac389523b2b4c25fc7190d6
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built gputil
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[]
no GPU. Are you sure the hardware accelerator is configured to GPU? To do this go to Runtime→Change runtime type and change the Hardware accelerator to GPU.


Configurez votre mode d'exécution du Google Colab en mode GPU.

> To use the google colab in a GPU mode you have to make sure the hardware accelerator is configured to GPU. To do this go to Runtime→Change runtime type and change the Hardware accelerator to GPU. Sometimes, all GPUs are in use and there is no GPU available.

Une fois configuré le GPU, vérifiez l'état de la mémoire sur la carte en réexécutant le code "memory footprint" ci-dessus. Le message a-t-il changé favorablement ?

Le mode d'exécution GPU est nécessaire pour le fine-tuning de BERT ci-dessous. 

# Allociné dataset

The [Allociné dataset](https://huggingface.co/datasets/allocine) is a French-language dataset for sentiment analysis. The texts are movie reviews written between 2006 and 2020 by members of the Allociné.fr community for various films. It contains 100k positive and 100k negative reviews divided into train (160k), validation (20k), and test (20k). 


### From huggingface datahub

After execution of this cell, you must restart the runtime in order to use  newly installed versions.

Just click on the button on the suggester "restart runtime" button (or go the runtime menu).

In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 5.2 MB/s 
     |████████████████████████████████| 95 kB 4.2 MB/s 
     |████████████████████████████████| 115 kB 58.5 MB/s 
     |████████████████████████████████| 163 kB 58.5 MB/s 
     |████████████████████████████████| 212 kB 18.1 MB/s 
     |████████████████████████████████| 127 kB 28.8 MB/s 
     |████████████████████████████████| 115 kB 56.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


Retrieve the [allociné dataset](https://huggingface.co/datasets/allocine) from the huggingface hub.

In [3]:
# https://huggingface.co/datasets/allocine

from datasets import load_dataset_builder
ds_builder = load_dataset_builder("allocine")

# Inspect dataset description
print(ds_builder.info.description)
print("========== DESCRIPTION ===============")

# Inspect dataset features
print(ds_builder.info.features)
print("========== FEATURES ===============")

# get_dataset_split_names
from datasets import get_dataset_split_names
get_dataset_split_names("allocine")

# load_dataset
from datasets import load_dataset
allocine_dataset = load_dataset("allocine")
allocine_dataset
#train_dataset = load_dataset("allocine", split="train")

 Allocine Dataset: A Large-Scale French Movie Reviews Dataset.
 This is a dataset for binary sentiment classification, made of user reviews scraped from Allocine.fr.
 It contains 100k positive and 100k negative reviews divided into 3 balanced splits: train (160k reviews), val (20k) and test (20k).

========== DESCRIPTION ===============
{'review': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}
========== FEATURES ===============


Generating train split:   0%|          | 0/160000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Dataset allocine downloaded and prepared to /root/.cache/huggingface/datasets/allocine/allocine/1.0.0/ea86b1dc05eae3a45a07b6281f2d4033b5fe7927b1008d06aa457ca1eae660d0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['review', 'label'],
        num_rows: 160000
    })
    validation: Dataset({
        features: ['review', 'label'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['review', 'label'],
        num_rows: 20000
    })
})

#### Select data ratio to process and create Pandas DataFrame

Definition of a method to select a ratio of a dataset split. By default return 100 % of the split.

In [4]:
import numpy as np

def get_dataset_split_ratio(dataset_split, seed=42, ratio=100):
  if ratio > 100 or ratio < 1: ratio = 100
  reviews = dataset_split['review']
  labels = dataset_split['label']
  
  data = [pair for pair in zip(reviews, labels)]
  
  np.random.seed(seed)
  np.random.shuffle(data)
  
  ratio_len = int(len(data)*ratio/100)
  
  data = list(zip(*data))

  return {'review':data[0][:ratio_len], 'label':data[1][:ratio_len]}


Definition of the method to create a dataframe from a huggingface dataset split (`{'review':[], 'label':[]}`) :

In [5]:
import pandas as pd

# From huggingface dataset split to DataFrame
def hf_dataset_split_to_df (huggingface_dataset_split):
  df = pd.DataFrame(huggingface_dataset_split)
  # https://github.com/amaiya/ktrain/blob/master/examples/text/ArabicHotelReviews-nbsvm.ipynb
  df['label'] = df['label'].apply(lambda x: 'negative' if x == 0 else 'positive')
  df = pd.concat([df, df.label.astype('str').str.get_dummies()], axis=1, sort=False)
  df = df[['review', 'negative', 'positive']]
  return df

Sélectionner la taille des données sur lesquelles vous voulez travailler et construire la dataframe correspondante.

Le code ci-dessous extrait 5 % du corpus. C'est suffisant pour étudier comment fonctionne les modèles présentés ci-après sans attendre trop de temps d'entraînement.

In [6]:
#ratio = 100 # percent of the data
ratio = 50

train_df = hf_dataset_split_to_df(get_dataset_split_ratio(allocine_dataset['train'], 42, ratio))
val_df = hf_dataset_split_to_df(get_dataset_split_ratio(allocine_dataset['validation'], 42, ratio))
test_df = hf_dataset_split_to_df(get_dataset_split_ratio(allocine_dataset['test'], 42, ratio))

print(train_df.head())
print(train_df.describe())
print('len(train_df) :', len(train_df))
print('len(val_df) :', len(val_df))
print('len(test_df) :', len(test_df))

                                              review  negative  positive
0  Un excellent thriller d'action où les scènes d...         0         1
1  Si le scénariste, qui aurait pu faire un minim...         1         0
2  Référence dans la filmographie de Bogart, "Le ...         1         0
3  Un bon scénario, un bon film, une histoire lou...         0         1
4  Un scenario vide et une mise en scene trés sop...         1         0
           negative      positive
count  80000.000000  80000.000000
mean       0.495000      0.505000
std        0.499978      0.499978
min        0.000000      0.000000
25%        0.000000      0.000000
50%        0.000000      1.000000
75%        1.000000      1.000000
max        1.000000      1.000000
len(train_df) : 80000
len(val_df) : 10000
len(test_df) : 10000


# ktrain

[ktrain](https://github.com/amaiya/ktrain) a lightweight wrapper for the deep learning library _TensorFlow Keras_ (and other libraries) to help build, train, and deploy neural networks and other machine learning models. Inspired by ML framework extensions like fastai and ludwig, ktrain is designed to make deep learning and AI more accessible and easier to apply for both newcomers and experienced practitioners.


## Configuration et installation de ktrain

Configuration de l'environnement

In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 

Installation de la bibliothèque ktrain

In [8]:
!pip install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.3 MB 1.4 MB/s 
     |████████████████████████████████| 981 kB 34.7 MB/s 
     |████████████████████████████████| 263 kB 46.4 MB/s 
     |████████████████████████████████| 3.8 MB 42.8 MB/s 
     |████████████████████████████████| 1.3 MB 48.4 MB/s 
     |████████████████████████████████| 468 kB 10.5 MB/s 
     |████████████████████████████████| 880 kB 45.6 MB/s 
     |████████████████████████████████| 7.6 MB 40.9 MB/s 
  Created wheel for ktrain: filename=ktrain-0.31.10-py3-none-any.whl size=25312982 sha256=d73c2212c6b29388de8251f0c544407e4265c1b0a95457a3d0d8c14a390dcf56
  Stored in directory: /root/.cache/pip/wheels/9c/1c/1b/6df2db85720b8f5c6ea5e3ae37313cfc656f248abf910b7cfd
  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33517 sha256=e5c7ba4faa4034d79d0e4bd5408611543dcc74d8746f8759a2d6a20447596e8e
  Stored in dire

Import de la bibliothèque ktrain dédié aux traitements de la modalité textuelle

In [9]:
# Execution time 30 s
import ktrain
from ktrain import text

## Fonctionnement de ktrain


Pour réaliser cette tâche de classification nous allons utiliser la bibliothèque ktrain pour faire un apprentissage par transfert.

Les étapes sont les suivantes
1. chargement des données avec application d'un prétraitement défini à la volée
2. construction d'un modèle de classification sur la base d'un modèle pré-entraîné spécifié
3. récupération d'une instance du modèle pour la personnalisation de celui-ci
4. recherche d'un bon taux d'apprentissage
5. entraînement du classifieur i.e. personnalisation du modèle de base à l'aide d'un taux d'apprentissage défini
6. utilisation du nouveau modèle

*ETAPE 1 :* 

Le type de pré-traitement est fonction du modèle pré-entraîné spécifié.

### Preprocessing
1. Lowercase
2. Remove multiple spaces
3. Remove stop words
3. Stemming with porterStemmer

In [10]:
train_data = list()
for elt in train_df['review']:
  train_data.append(elt)

In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
def porter_stemmer(review):
  ps = PorterStemmer()
  return ps.stem(review)

def preprocessing(train_data):
  train = list()
  for review in train_data:
    review = review.lower()
    review_tokens = word_tokenize(review)
    review_without_sw = [word for word in review_tokens if not word in stopwords.words()]
    review = (" ").join(review_without_sw)
    
    review = ' '.join(review.split()) # replace multiple whitespace by a single one
    review = porter_stemmer(review)
    train.append(review) 
  
  return train


In [ ]:
train_data = preprocessing(train_data)

print(train_data[:10])

*ETAPE 2 :*

ktrain vient avec quelques modèles pré-entraînés packagés. Pour les connaître, exécutez : 

In [ ]:
text.print_text_classifiers()

Vous connaissez **[fasttext](https://github.com/facebookresearch/fastText)** de (Facebook/Meta) car vous l'avez déjà utilisé dans un [précédent TP](https://github.com/nicolashernandez/teaching_nlp/blob/main/04_repr%C3%A9sentation_vectorielle_continue.ipynb)... fasttext permet de produire une représentation continue des mots en suivant une approche à la word2vec. Sa spécificité est qu'il propose de traiter la variabilité morphologique des mots en construisant des vecteurs non pas pour des mots mais pour des sous-mots (séquence de caractères). Le vecteur d'un mot est la somme de tous les vecteurs des sous-mots le composant. Cette approche est indépendante de la langue, et montre de meilleurs résultats que word2vec sur des tâches syntaxiques, surtout quand le corpus d'entraînement est petit. Word2vec est légèrement meilleur pour des tâches sémantiques. Un des avantage de FastText est de pouvoir fournir des vecteurs mêmes pour les mots hors vocabulaires.



[**NBSVM**](https://medium.com/@asmaiya/a-neural-implementation-of-nbsvm-in-keras-d4ef8c96cb7c) is an approach to text classification proposed by [Wang and Manning](https://www.aclweb.org/anthology/P12-2018) that takes a linear model such as SVM (or logistic regression) and infuses it with Bayesian probabilities by replacing word count features with Naive Bayes log-count ratios. Despite its simplicity, NBSVM models have been shown to be both fast and powerful across a wide range of different text classification datasets. 
Keras offers a NBSVM model implemented as a neural network using two embedding layers. The first stores the Naive Bayes log-count ratios. The second stores learned weights (or coefficients) for each feature (i.e., word) in this linear model. The prediction, then, is simply the dot product of these two vectors.

**BERT** (_Bidirectional Encoder Representations from Transformers_), proposé par [Google AI Language](https://arxiv.org/pdf/1810.04805.pdf) est un encodeur bidirectionnel qui applique un modèle d'attention Transformers à la modélisation du language (_language modeling_). Il représente l'état de l'art.
Les Transformers sont un mécanisme d'attention qui apprennent les relations contextuelles entre les mots dans un texte.


Il est possible d'[**encapsuler les modèles Transformers du site _hugging face_**](https://github.com/amaiya/ktrain/blob/master/tutorials/tutorial-A3-hugging_face_transformers.ipynb). _hugging face_ diffuse les [modèles Transformers pré-entraînés "officiels"](https://huggingface.co/transformers/pretrained_models.html) ainsi que les [modèles Transformers construits par la communauté](https://huggingface.co/models).


*ETAPE 4 :*


Le **taux d'apprentissage (_learning rate_)** est un hyperparamètre qui contrôle combien le modèle doit changer en réponse à l'erreur estimée à chaque fois que les poids du modèles sont mis à jour. Choisir un 'lr' trop petit conduit à une longue phase d'entraînement qui peut resté bloquée. Choisir un 'lr' trop grand conduit à un apprentissage sous-optimal des poids et à une instabilité du processus d'entraînement. 

  

## FastText



*ETAPE 1 :* 

Les méthodes [`texts_from_csv`](https://amaiya.github.io/ktrain/text/index.html#ktrain.text.texts_from_csv) ou [`texts_from_df`](https://amaiya.github.io/ktrain/text/index.html#ktrain.text.texts_from_df) charge le corpus, normalise les documents (définit un préprocesseur réutilisable), et découpe la collection en données d'entraînement et données de validation (à moins que des données de validation soient passées en argument).

Cette étape est commune aux modèles FastText et NBSVM (`preprocess_mode='standard'`). BERT utilise son propre tokenizer. Il faudra faire un pré-traitement dédié (`preprocess_mode='bert'`).

```
* train_filepath(str): file path to training CSV
* text_column(str): name of column containing the text
* label_column(list): list of columns that are to be treated as labels
* val_filepath(string): file path to test CSV.  If not supplied, 10% of documents in training CSV will be used for testing/validation.
* max_features(int): max num of words to consider in vocabulary ; Note: This is only used for preprocess_mode='standard'.
* maxlen(int): each document can be of most <maxlen> words. 0 is used as padding ID.
* ngram_range(int): size of multi-word phrases to consider e.g., 2 will consider both 1-word phrases and 2-word phrases limited by max_features
* preprocess_mode (str):  Either 'standard' (normal tokenization) or one of {'bert', 'distilbert'} tokenization and preprocessing for use with                BERT/DistilBert text classification model.
```



        
  

In [ ]:
# fasttext
#NUM_WORDS = 50000
#MAXLEN = 150
#NGRAMS_SIZE = 1# 1 # 8 minutes avec 2 pour 10000 examples

# nbsvm 
#NUM_WORDS = 80000
#MAXLEN = 2000
#NGRAMS_SIZE = 3

(x_train_preproc, y_train_preproc), (x_val_preproc, y_val_preproc), preproc = text.texts_from_df (train_df, 
                                                                   'review', # name of column containing review text
                                                                   label_columns=['negative', 'positive'],
                                                                   val_df=val_df, # if None, 10% of data will be used for validation
                                          #max_features=NUM_WORDS, 
                                          #maxlen=MAXLEN,
                                          #ngram_range=NGRAMS_SIZE,
                                          preprocess_mode='standard' # default
                                          )



Observons les 5 premières des données prétraitées. `x_` représente la donnée et `y_` la classe. On note que les données ont été transformées. Chaque mot est remplacé par un identifiant. On note que la classe est décrite par 2 colonnes avec deux codes "1 0" et "0 1".

In [ ]:
print ('x_train_preproc', x_train_preproc[:5])
print ('y_train_preproc', y_train_preproc[:5])

*ETAPE 2 et 3:* 


In [ ]:
# Build and return a text classification model https://amaiya.github.io/ktrain/text/index.html#ktrain.text.text_classifier
fasttext_model = text.text_classifier('fasttext', (x_train_preproc, y_train_preproc), preproc=preproc)

# Returns a Learner instance that can be used to tune and train Keras models https://amaiya.github.io/ktrain/index.html#ktrain.get_learner
fasttext_learner = ktrain.get_learner(fasttext_model, train_data=(x_train_preproc, y_train_preproc), val_data=(x_val_preproc, y_val_preproc))

*ETAPE 4 :* 

La méthode [`lr_find`](https://amaiya.github.io/ktrain/core.html#ktrain.core.Learner.lr_find) implémente la méthode [*Cyclical Learning Rates*](https://arxiv.org/abs/1506.01186) qui permet d'estimer un lr sans une exploration systématique d'un lr qui décroit. L'approche peut prendre seulement le temps de quelques itérations (i.e. du traitement de quelques batchs) sans même tourner sur plusieurs époques. Pour rappel, une époque correspond au traitement de l'ensemble des données d'entrainements par lot/batch d'exemples. *For example,
CIFAR-10 has 50, 000 training images and the batchsize is
100 so an epoch = 50, 000/100 = 500 iterations*. 

Then plots loss as learning rate is increased.

*Highest learning rate corresponding to a still falling loss should be chosen*. "[The point of optimality [...] is the point on the training rate plot where the slope is steepest in the downwards direction. That's because this plot shows the loss after one epoch. Points before the steepest slope are training too slowly. Points after the steepest slope are at risk of training too quickly: usually, but not always (it didn't happen in this demo case), they will fall off the mountain in terms of loss because they jump past the point of optimality](https://www.kaggle.com/code/residentmario/finding-an-optimal-learning-rate-with-lr-finder)".

In [ ]:
# recherche d'un bon taux d'apprentissage 
# you can set max_epochs (e.g., max_epochs=5) to estimate LR
fasttext_learner.lr_find() # 
fasttext_learner.lr_plot()

*ETAPE 5 :* 

[autofit](https://amaiya.github.io/ktrain/core.html#ktrain.core.Learner.autofit)
Automatically train model using a default learning rate schedule shown to work well in practice.  By default, this method currently employs a triangular learning rate policy (https://arxiv.org/abs/1506.01186).  
During each epoch, this learning rate policy varies the learning rate from lr/10 to lr and then back to a low learning rate that is near-zero. 
If epochs is None, then early_stopping and reduce_on_plateau are atomatically
set to 6 and 3, respectively.


#### QUESTION : training

* Sur le graph ci-dessus, repérez approximativement la puissance `n` de `1/10^n` où la chute de la loss devient importante. Testez avec cette valeur comme learning rate, observez votre performance (accuracy) sur le train et le val.

Dans l'extrait de log ci-dessous, loss et accuracy concerne le corpus de train tandis que  val_loss et val_accuracy le corpus de validation (sur un corpus sample).
```
Epoch 12/1024
244/250 [============================>.] - ETA: 0s - loss: 0.1109 - accuracy: 0.9589
Restoring model weights from the end of the best epoch: 7.
250/250 [==============================] - 1s 6ms/step - loss: 0.1108 - accuracy: 0.9589 - val_loss: 0.3864 - val_accuracy: 0.8820
Epoch 12: early stopping
```
* Est-ce normal d'observer un écart d'accuracy entre le train et le val ?
* Stoquez le score obtenu en dernière étape, et relancez le finetuning sans changer le paramétrage. Obtenez-vous les mêmes résultats ? Pourquoi ? La réponse est à chercher dans l'initiatilisation des poids du réseau neuronal.
* Volontairement (et selon le temps qu'a pris votre entraînement) testez des lr avec d'autres puissances de 10 (0.1, 0.01, 0.001, 0.0001). Attention les lr les plus petites prendront le plus de temps !!! Est-ce que cela marche mieux ? Attention, suivant votre choix de lr l'entraînement peut prendre quelques minutes à au moins 1h....

Mon meilleur score est `loss: 0.1108 - accuracy: 0.9589 - val_loss: 0.3864 - val_accuracy: 0.8820` sur le corpus sample et vous ?

In [56]:
#LEARNING_RATE = 0.01
LEARNING_RATE = [0.01, 0.001, 0.02, 0.002, 0.003, 0.005, 0.5]
#fasttext_learner.autofit(LEARNING_RATE)
for lr in LEARNING_RATE:
  print("=================== TRAINING WITH LR: ", lr, "===================")
  fasttext_learner.autofit(lr)

# Epoch 16/1024
# 4491/4500 [============================>.] - ETA: 0s - loss: 0.1540 - accuracy: 0.9418Restoring model weights from the end of the best epoch: 11.
# 4500/4500 [==============================] - 25s 6ms/step - loss: 0.1539 - accuracy: 0.9418 - val_loss: 0.2173 - val_accuracy: 0.9206
# Epoch 16: early stopping

# wi all allocine training dataset (at least one hour run) 
#learner.autofit(0.00001)
# Epoch 129/1024
# 4497/4500 [============================>.] - ETA: 0s - loss: 0.2197 - accuracy: 0.9135Restoring model weights from the end of the best epoch: 124.
# 4500/4500 [==============================] - 29s 6ms/step - loss: 0.2197 - accuracy: 0.9135 - val_loss: 0.2058 - val_accuracy: 0.9204
# Epoch 129: early stopping

=================== TRAINING WITH LR:  0.01 ===================
early_stopping automatically enabled at patience=5
reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 0.01...
Epoch 1/1024
2500/2500 [==============================] - 16s 6ms/step - loss: 0.4227 - accuracy: 0.8065 - val_loss: 0.2472 - val_accuracy: 0.9022
Epoch 2/1024
2500/2500 [==============================] - 17s 7ms/step - loss: 0.2984 - accuracy: 0.8763 - val_loss: 0.2289 - val_accuracy: 0.9116
Epoch 3/1024
2500/2500 [==============================] - 15s 6ms/step - loss: 0.2727 - accuracy: 0.8900 - val_loss: 0.2215 - val_accuracy: 0.9127
Epoch 4/1024
2500/2500 [==============================] - 16s 6ms/step - loss: 0.2561 - accuracy: 0.8970 - val_loss: 0.2201 - val_accuracy: 0.9134
Epoch 5/1024
2500/2500 [==============================] - 16s 6ms/step - loss: 0.2447 - accuracy: 0.9030 - val_loss: 0.2222 - val_accuracy: 0.9112
Epoch 6/1024
2494/

#### VOTRE REPONSE

**TODO**

save/reload a model

https://github.com/amaiya/ktrain/blob/master/FAQ.md#how-do-i-resume-training-from-a-saved-checkpoint

In [ ]:
# save Predictor (i.e., model and Preprocessor instance) after partially training
ktrain.get_predictor(fasttext_learner.model, preproc).save('fasttext_allocine.model+preproc')

# reload Predictor and extract model
#model = ktrain.load_predictor('/tmp/my_predictor').model

*ETAPE 6 :* Le code ci-dessous permet d'utiliser le modèle. Notez que le paramétrage `return_proba=True` permet d'obtenir les probas... 

In [ ]:
fasttext_predictor = ktrain.get_predictor(fasttext_learner.model, preproc)

data = [ "Ce film était horrible ! L'intrigue était ennuyeuse. Le jeu d'acteur était correct, cependant.",
         "Le film est vraiment nul. Je veux qu'on me rende mon argent.",
        "Quelle belle comédie romantique. 10/10 à revoir !"]

# Makes predictions for a list of strings where each string is a document or text snippet.
print (fasttext_predictor.predict(data))

# If return_proba is True, returns probabilities of each class.
print (fasttext_predictor.predict(data,  return_proba=True))

#### QUESTION : évaluation qualitative légère

* Tester le modèle. Arrivez-vous à piéger le modèle ? Avec quelle phrase (donnez le code)

#### VOTRE REPONSE

**TODO**

### QUESTION : Evaluation quantitative

Génération d'une hypothèse pour les *review* du corpus de test (`list(test_df['review'])`)

In [ ]:
# reviews 
x_test = list(test_df['review'])

# labels (gold) 
y_test = list(test_df['positive'])

In [ ]:
# hypothèse
y_hyp = [0 if h == 'negative' else 1 for h in fasttext_predictor.predict(x_test) ]



[SemEval 2022 isarcasmeval task](https://sites.google.com/view/semeval2022-isarcasmeval) used the F1-score for the sarcastic class. sklearn permet de spécifier cette mesure avec le paramètre `average = 'binary'` éventuellement accompagné du paramètre qui indique la classe à considérer `pos_label = 1`. This metric should not be confused with the regular macro-F1. sklearn permet de calculer un score macro à l'aide de la valeur `average = 'macro'`.
Sans paramétrage c'est le F1-score d'une classe qui est retourné. Les performances rapportées par [Theophile Blart utilise aussi ce paramétrage par défaut](https://github.com/TheophileBlard/french-sentiment-analysis-with-bert).

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

f1_positive = f1_score(y_test, y_hyp, average = "binary", pos_label = 1)
f1_positive
#OR
#p_score_positive = precision_score(y_test, y_hyp, average = "binary", pos_label = 1)
#r_score_positive = recall_score(y_test, y_hyp, average = "binary", pos_label = 1)
#f1_positive = (2*p_score_positive*r_score_positive)/(p_score_positive +r_score_positive)

# 5 %   0.8922764227642277
# 100 % 0.9181408981471789

NameError: ignored

[Theophile Blard rapporte des performances avec différents modèles (CamemBERT, RNN, TF-IDF + LogReg, CNN, fastText (unigrams)) sur le dataset Allociné](https://github.com/TheophileBlard/french-sentiment-analysis-with-bert). Comment vous positionnez-vous par rapport à ses résultats ?   

## NBSVM

Le code ci-dessous utilise le même pré-traitement que précédemment et applique un modèle neuronal plus "simple" que fasttext.  

In [ ]:
# load an NBSVM model
nbsvm_model = text.text_classifier('nbsvm', (x_train_preproc, y_train_preproc), preproc=preproc)
nbsvm_learner = ktrain.get_learner(nbsvm_model, train_data=(x_train_preproc, y_train_preproc), val_data=(x_val_preproc, y_val_preproc))

# fine tune
LEARNING_RATE = 0.01
nbsvm_learner.autofit(LEARNING_RATE)
# Epoch 6/1024
#4498/4500 [============================>.] - ETA: 0s - loss: 0.1264 - accuracy: 0.9544Restoring model weights from the end of the best epoch: 1.
#4500/4500 [==============================] - 24s 5ms/step - loss: 0.1264 - accuracy: 0.9544 - val_loss: 0.2677 - val_accuracy: 0.9161
#Epoch 6: early stopping

# Finally, we will fit our model using and [SGDR learning rate](https://github.com/amaiya/ktrain/blob/master/example-02-tuning-learning-rates.ipynb) schedule by invoking the fit method with the cycle_len parameter (along with the cycle_mult parameter).
# learner.fit(0.001, 3, cycle_len=1, cycle_mult=2)


évaluation

In [ ]:
# données
x_test = list(test_df['review']) # reviews 
y_test = list(test_df['positive']) # labels (gold) 

# prédiction
nbsvm_predictor = ktrain.get_predictor(nbsvm_learner.model, preproc)
y_hyp = [0 if h == 'negative' else 1 for h in nbsvm_predictor.predict(x_test) ]

# évaluation
print(f1_score(y_test, y_hyp))

# 5   % 0.903981180311341
# 100 % 0.9203585411085926


save/reload a model

https://github.com/amaiya/ktrain/blob/master/FAQ.md#how-do-i-resume-training-from-a-saved-checkpoint

In [ ]:
# save Predictor (i.e., model and Preprocessor instance) after partially training
ktrain.get_predictor(nbsvm_learner.model, preproc).save('nbsvm_allocine.model+preproc')

# reload Predictor and extract model
#model = ktrain.load_predictor('/tmp/my_predictor').model

#### QUESTION

* Le modèle nbsvm est-il plus performant que le précédent sur les données de validations ? Vous pouvez tester aussi différents learning rate.

## BERT 

L'usage du modèle bert requiert que l'on change le prétraitement des données en entrée. Le code suivant réalise le prétraitement, charge un modèle bert et lance la personnalisation (fine tuning) sur 1 cycle avec taux d'apprentissage fixé.


In [ ]:
# ETAPE 1 
(x_train_preproc, y_train_preproc), (x_val_preproc, y_val_preproc), preproc = text.texts_from_df (train_df, 
                      'review',
                      label_columns = ["negative", "positive"],
                      val_df= val_df, # if None, 10% of data will be used for validation
                      ##max_features=NUM_WORDS, 
                      #maxlen=MAXLEN,
                      preprocess_mode='bert' 
                      )

['negative', 'positive']
   negative  positive
0         0         1
1         1         0
2         1         0
3         0         1
4         1         0
['negative', 'positive']
   negative  positive
0         0         1
1         0         1
2         1         0
3         1         0
4         1         0
downloading pretrained BERT model (multi_cased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: fr


Is Multi-Label? False
preprocessing test...
language: fr


#### QUESTION
* Exécutez le code en observant l'occupation de la RAM et attendez jusqu'à voir le temps prévisionnel s'afficher. Etes-vous choqué par le temps affiché ? BERT est très gros. Il requiert un peu de temps... Sur l'extrait le temps devrait être de 15 minutes à quelques heures sur le corpus complet.
* Si vous êtes en phase d'exploration et que vous ne travaillez pas sur un extrait des données, stoppez l'exécution dans la cellule, et tentez de changer la taille du batch_size (quantité de données traitées en même temps). Vous pouvez tester 12 et éventuellement 128... comme valeurs. Ne lachez pas la barre de la RAM des yeux. Ça passe ? Quel problème rencontrez-vous ? 
* Reprenez... et passez à la suite...

In [ ]:
# ETAPE 2 et 3
bert_model = text.text_classifier('bert', (x_train_preproc, y_train_preproc) , preproc=preproc)
bert_learner = ktrain.get_learner(bert_model, 
                             train_data=(x_train_preproc, y_train_preproc), 
                             val_data=(x_val_preproc, y_val_preproc), 
                             batch_size=6)
# ETAPE 5
bert_learner.fit_onecycle(2e-5, 1)

# Is Multi-Label? False
# maxlen is 400
# begin training using onecycle policy with max lr of 2e-05...
# 26667/26667 [==============================] - 18569s 695ms/step - loss: 0.1856 - accuracy: 0.9256 - val_loss: 0.1116 - val_accuracy: 0.9588
# <keras.callbacks.History at 0x7f72f1272b10>
# Exécution ~ 5h

Is Multi-Label? False
maxlen is 400
done.


begin training using onecycle policy with max lr of 2e-05...
26667/26667 [==============================] - 18569s 695ms/step - loss: 0.1856 - accuracy: 0.9256 - val_loss: 0.1116 - val_accuracy: 0.9588


évaluation

In [ ]:
# données
x_test = list(test_df['review'])   # reviews 
y_test = list(test_df['positive']) # labels (gold) 

# prédiction
bert_predictor = ktrain.get_predictor(bert_learner.model, preproc)
y_hyp = [0 if h == 'negative' else 1 for h in bert_predictor.predict(x_test) ]

# évaluation
from sklearn.metrics import f1_score, precision_score, recall_score
print(f1_score(y_test, y_hyp))
# 100 % 0.9581906836808797

0.9581906836808797


save/reload a model

https://github.com/amaiya/ktrain/blob/master/FAQ.md#how-do-i-resume-training-from-a-saved-checkpoint

In [ ]:
# save Predictor (i.e., model and Preprocessor instance) after partially training
ktrain.get_predictor(bert_learner.model, preproc).save('bert_allocine.model+preproc')

# reload Predictor and extract model
#model = ktrain.load_predictor('/tmp/my_predictor').model

## Modèle "à la bert" issu de hugging face

De nombreux [modèles sont disponibles sur HuggingFace](https://huggingface.co/models). 

On va tester un modèle plus léger que BERT à savoir 'distilbert-base-uncased'. 

In [ ]:
x_train = list(train_df['review'])
y_train = list(train_df['positive'])

x_val = list(val_df['review'])
y_val = list(val_df['positive'])

x_test = list(test_df['review'])
y_test = list(test_df['positive'])

## Do not consider the following

## Convert a list of string into lists of int then into np.array.
## This step should be skiped when running BERT

#import numpy as np
#y_train = np.array(list(map(int, y_train)))
#y_val = np.array(list(map(int, y_val)))
#y_test = np.array(list(map(int, y_test)))

#print ('x_train', len(x_train), x_train[:10])
#print ('y_train',  len(y_train), y_train[:10])
#print ('x_val', len(x_val), x_val[:10])
#print ('y_val',  len(y_val), y_val[:10])
#print ('x_test', len(x_test))
#print ('y_test',  len(y_test))

In [ ]:
import ktrain
from ktrain import text
MODEL_NAME = 'distilbert-base-uncased'
#MODEL_NAME = 'albert-base-v2'
#MODEL_NAME = 'camembert-base'

CLASS_NAMES = ["negative", "positive"]

distilbert_preproc = text.Transformer(MODEL_NAME, maxlen=500, class_names=CLASS_NAMES)
train_preproc = distilbert_preproc.preprocess_train(x_train, y_train)
val_preproc = distilbert_preproc.preprocess_test(x_val, y_val)
#print (type(trn))
#x_train, y_train = trn
#x_test, y_test = val  
distilbert_model = distilbert_preproc.get_classifier()
# batch_size (int):              Batch size to use in training. default:32  
#learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6) # 128 dépend de la ram dispo 13 Go par défaut
distilbert_learner = ktrain.get_learner(distilbert_model, train_data=train_preproc, val_data=val_preproc, batch_size=12)

In [ ]:
#del distilbert_preproc
#del train_preproc
#del val_preproc
#del distilbert_model
#del distilbert_learner

In [ ]:
distilbert_learner.fit_onecycle(0.01, 1)
#learner.fit_onecycle(8e-5, 4)
#8e-5 = 8 + 10^(-5)

In [ ]:
# données
x_test = list(test_df['review'])   # reviews 
y_test = list(test_df['positive']) # labels (gold) 

# prédiction
distilbert_predictor = ktrain.get_predictor(distilbert_learner.model, distilbert_preproc)
y_hyp = [0 if h == 'negative' else 1 for h in distilbert_predictor.predict(x_test) ]

# évaluation
print(f1_score(y_test, y_hyp))
# distilbert-base-uncased train sur 5 % 0.6577181208053691
# distilbert-base-uncased train sur 100 % 0.6482833198161666


0.6482833198161666


backup model

In [ ]:
import pickle
with open('distilbert_allocine.model', 'wb') as model_file:
  pickle.dump(distilbert_learner, model_file)

#### QUESTION
* Quelle performance obtenez-vous avec BERT et distilBERT pour quel temps d'entraînement ? Quelle performance comparativement aux 2 modèles précédents nbsvm et fasttext ?
* Quelles risques prenez-vous à ne pas réaliser les mêmes prétraitements ou normalisations (voire utiliser des outils différents pour réaliser les mêmes prétraitements ou normalisations supposées) que ceux réalisés sur les corpus ayant servis à construire les modèles ? 
* En résumé, en mettant dans la balance les questions de performance, les questions de taille de modèles, de temps de "fine-tuning"... quelles conclusions faites-vous de l'usage des modèles simples vs les modèles plus complexes à la BERT ?

Si le code tourne toujours passez à la question suivante pour gagner du temps...

#### VOTRE REPONSE

**TODO**


# Devoir à rendre 
[Theophile Blard rapporte des performances avec différents modèles (CamemBERT, RNN, TF-IDF + LogReg, CNN, fastText (unigrams)) sur le dataset Allociné](https://github.com/TheophileBlard/french-sentiment-analysis-with-bert). 


| Model                                        | Validation Accuracy | Validation F1-Score | Test Accuracy | Test F1-Score |
| :--------------------------------------------|--------------------:| -------------------:| -------------:|--------------:|
| **[CamemBERT][bert.ipynb]**                  |           **97.39** |           **97.36** |     **97.44** |     **97.34** |
| [RNN]                    |               94.39 |               94.34 |         94.58 |         94.39 |
| [TF-IDF + LogReg]              |               94.35 |               94.29 |         94.38 |         94.19 |
| [CNN]                    |               93.69 |               93.72 |         94.10 |         93.98 |
| [fastText (unigrams)]    |               92.88 |               92.75 |         92.90 |         92.57 |


Arriverez-vous à faire mieux ?

Bien entendu vous avez le droit de choisir les modèles et leur configuration.

Explorez les thèmes suivants
- data augmentation
- model ensembling 

Ecrivez un rapport d'expériences.

## Topics

### Data augmentation in NLP
TextAttack is a Python framework for adversarial attacks, adversarial training, and [data augmentation in NLP](https://textattack.readthedocs.io/en/latest/2notebook/3_Augmentations.html).

Here some hints for [multi-language attacks](https://textattack.readthedocs.io/en/latest/2notebook/Example_4_CamemBERT.html)

### Model ensembling 

- Train/fine-tune multiple models from various architectures and/or dataset folds
- Then ensemble them by (hard/soft) voting mechanism or something else...



## VOTRE REPONSE

**TODO**

# Références
* Text Classification Example: Sentiment Analysis with IMDb Movie Reviews¶ https://nbviewer.org/github/amaiya/ktrain/blob/master/tutorials/tutorial-04-text-classification.ipynb
* https://nbviewer.org/github/amaiya/ktrain/blob/master/examples/text/IMDb-BERT.ipynb
* ktrain examples of Binary text Classification (Sentiment Analysis with IMDb Movie Reviews) with a nbsvm model (also a bit of bert)  and 
Multi-Label Text Classification (toxic comments) with fasttext https://nbviewer.org/github/amaiya/ktrain/blob/master/tutorials/tutorial-04-text-classification.ipynb
* Text Classification with Hugging Face Transformers in ktrain https://github.com/amaiya/ktrain/blob/master/tutorials/tutorial-A3-hugging_face_transformers.ipynb
* ktrain api documentation https://amaiya.github.io/ktrain/
* Evaluation of various models (CamemBERT, RNN, TF-IDF + LogReg, CNN, fastText (unigrams)) on allocine dataset https://github.com/TheophileBlard/french-sentiment-analysis-with-bert
* The Allociné dataset is a French-language dataset for sentiment analysis. The texts are movie reviews written between 2006 and 2020 by members of the Allociné.fr community for various films. It contains 100k positive and 100k negative reviews divided into train (160k), validation (20k), and test (20k). 
https://huggingface.co/datasets/allocine
* Amazon reviews for three product categories: books, DVD, and music. Each sample contains a review text and the associated rating from 1 to 5 stars. Reviews rated above 3 is labeled as positive, and those rated less than 3 is labeled as negative. https://github.com/getalp/Flaubert/tree/master/flue
* Twitter API between May and September 2018. The sentiment was generated thanks to AWS Comprehend API. For Spanish and French, tweets were first translated to English using Google Translate, and then analyzed with AWS Comprehend. https://github.com/charlesmalafosse/open-dataset-for-sentiment-analysis/
* French dataset for sentiment analysis (Data translated from English to French).  A collection of over 1.5 Million tweets data translated to French, with their sentiment. The data has two columns, polarity and status https://github.com/gamebusterz/French-Sentiment-Analysis-Dataset
* English Classification datasets https://ludwig.ai/latest/examples/text_classification/